###### Imports

In [214]:
import requests
import json
import pandas as pd

In [215]:
pd.set_option('display.max_columns', None)

# This file outlines the prep for the 2010 to 2019 5 year ACS estimate values to be used in a linear regression for finding peer communities in terms of our ageing population.

### First we will create API calls for the population and home value data from the Census Bureau ACS 1 year estimates.

###### The data we're curious about comes divided by sex or race - for now we're going to look at sex and then total them together because it's less data to deal with. You are only able to import 50 variables at once, so we're going to do total population and male population together, then female population, followed by home value. Eventually we will join them into one larger dataframe.

In [216]:
#male 2010-2019 and total

range = ['10','11','12','13','14','15','16','17','18','19']
api_key = '24fc7d81b74510d599f702dbd408fb18e1466d81'

male_appended = []
for i in range:
    url_str= "https://api.census.gov/data/20{}/acs/acs1?get=NAME,GEO_ID,B01001_001E,B01001_002E,B01001_003E,B01001_004E,B01001_005E,B01001_006E,B01001_007E,B01001_008E,B01001_009E,B01001_010E,B01001_011E,B01001_012E,B01001_013E,B01001_014E,B01001_015E,B01001_016E,B01001_017E,B01001_018E,B01001_019E,B01001_020E,B01001_021E,B01001_022E,B01001_023E,B01001_024E,B01001_025E&for=metropolitan%20statistical%20area/micropolitan%20statistical%20area:*&key=".format(i)+api_key
    predicates= {}
    get_vars= ["NAME","GEO_ID",'B01001_001E','B01001_020E','B01001_003E','B01001_004E','B01001_005E','B01001_006E','B01001_007E','B01001_008E','B01001_009E','B01001_010E','B01001_011E','B01001_012E','B01001_013E','B01001_014E','B01001_015E','B01001_016E','B01001_017E','B01001_018E','B01001_019E','B01001_020E','B01001_021E','B01001_022E','B01001_023E','B01001_024E','B01001_025E']
    predicates["get"]= ",". join(get_vars)
    predicates["for"]= "state:*"
    male= requests.get(url_str, params= predicates)
    
    col_names = ['CBSA','GEOID','total','mtotal','mu5',
                 'm5_9','m10_14','m15_17','m18_19','m20',
                 'm21','m22_24','m25_29','m30_34',
                 'm35_39','m40_44','m45_49','m50_54','m55_59',
                 'm60_61','m62_64','m65_66','m67_69',
                 'm70_74','m75_79','m80_84','m85+','ind']
    male=pd.DataFrame(columns=col_names, data=male.json()[1:])
    male['year'] = '20{}'.format(i)
    male_appended.append(male)
    
male_appended = pd.concat(male_appended)
male = male_appended
print('Done')

Done


In [217]:
male.year.unique()

array(['2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017',
       '2018', '2019'], dtype=object)

In [218]:
male.head()

,CBSA,GEOID,total,mtotal,mu5,m5_9,m10_14,m15_17,m18_19,m20,m21,m22_24,m25_29,m30_34,m35_39,m40_44,m45_49,m50_54,m55_59,m60_61,m62_64,m65_66,m67_69,m70_74,m75_79,m80_84,m85+,ind,year
0,"Aberdeen, WA Micro Area",310M100US10140,72882,37419,2177,1697,2600,1333,871,327,723,1399,2214,2269,2184,2680,2666,3104,3242,941,1440,653,936,1715,974,735,539,10140,2010
1,"Abilene, TX Metro Area",310M100US10180,164941,82694,5941,5059,5579,2942,3269,1390,1111,5685,6758,5375,4275,6111,5364,5418,5652,1426,2337,1716,1585,1775,1426,1204,1296,10180,2010
2,"Adrian, MI Micro Area",310M100US10300,99763,50213,2898,3360,3441,2211,1623,637,516,1905,3020,3153,3422,3175,4064,3838,4247,1150,1293,728,1137,1829,1059,844,663,10300,2010
3,"Aguadilla-Isabela-San Sebasti?n, PR Metro Area",310M100US10380,305988,149560,9529,10792,10016,6552,5445,2185,1854,5578,9276,9324,10716,9875,9928,9641,8462,4236,5309,3520,4598,5104,3160,2670,1790,10380,2010
4,"Akron, OH Metro Area",310M100US10420,702951,341630,20054,20805,24467,15115,11518,4644,5620,15959,20638,19911,19937,22907,25878,27269,24857,8941,10447,5069,8071,10656,8882,6239,3746,10420,2010


In [230]:
male2010.head()

,CBSA,GEOID,total,mtotal,mu5,m5_9,m10_14,m15_17,m18_19,m20,m21,m22_24,m25_29,m30_34,m35_39,m40_44,m45_49,m50_54,m55_59,m60_61,m62_64,m65_66,m67_69,m70_74,m75_79,m80_84,m85+,ind
0,"Aberdeen, WA Micro Area",310M100US10140,72882,37419,2177,1697,2600,1333,871,327,723,1399,2214,2269,2184,2680,2666,3104,3242,941,1440,653,936,1715,974,735,539,10140
1,"Abilene, TX Metro Area",310M100US10180,164941,82694,5941,5059,5579,2942,3269,1390,1111,5685,6758,5375,4275,6111,5364,5418,5652,1426,2337,1716,1585,1775,1426,1204,1296,10180
2,"Adrian, MI Micro Area",310M100US10300,99763,50213,2898,3360,3441,2211,1623,637,516,1905,3020,3153,3422,3175,4064,3838,4247,1150,1293,728,1137,1829,1059,844,663,10300
3,"Aguadilla-Isabela-San Sebasti?n, PR Metro Area",310M100US10380,305988,149560,9529,10792,10016,6552,5445,2185,1854,5578,9276,9324,10716,9875,9928,9641,8462,4236,5309,3520,4598,5104,3160,2670,1790,10380
4,"Akron, OH Metro Area",310M100US10420,702951,341630,20054,20805,24467,15115,11518,4644,5620,15959,20638,19911,19937,22907,25878,27269,24857,8941,10447,5069,8071,10656,8882,6239,3746,10420


In [227]:
male.tail()

,CBSA,GEOID,total,mtotal,mu5,m5_9,m10_14,m15_17,m18_19,m20,m21,m22_24,m25_29,m30_34,m35_39,m40_44,m45_49,m50_54,m55_59,m60_61,m62_64,m65_66,m67_69,m70_74,m75_79,m80_84,m85+,ind,year
513,"York-Hanover, PA Metro Area",310M500US49620,449058,222750,13064,14438,14519,9764,5346,2914,2198,8070,14203,14259,13685,13455,14214,15332,14759,6626,9564,5669,7306,9448,6222,4891,2804,49620,2019
514,"Youngstown-Warren-Boardman, OH-PA Metro Area",310M500US49660,536081,262932,15166,14144,16198,10209,7238,3395,4444,9692,16109,14445,12708,16686,15671,17498,17892,8484,12154,7112,10189,12899,8975,5760,5864,49660,2019
515,"Yuba City, CA Metro Area",310M500US49700,175639,88746,6388,7803,5738,4123,2731,594,797,4346,7570,5304,6318,4890,4431,5377,4428,2713,3569,1845,2029,3320,1849,1173,1410,49700,2019
516,"Yuma, AZ Metro Area",310M500US49740,213787,110189,7557,7491,7850,4188,3236,1199,1618,7454,8713,7797,6839,5594,5058,5675,4680,2555,3364,1916,2652,4670,4793,3612,1678,49740,2019
517,"Zanesville, OH Micro Area",310M500US49780,86215,42589,2462,2046,3512,1527,1027,862,1850,1040,2734,2226,2081,2618,3000,2881,3677,944,1053,867,1349,2178,1328,608,719,49780,2019


In [231]:
male2010.tail()

,CBSA,GEOID,total,mtotal,mu5,m5_9,m10_14,m15_17,m18_19,m20,m21,m22_24,m25_29,m30_34,m35_39,m40_44,m45_49,m50_54,m55_59,m60_61,m62_64,m65_66,m67_69,m70_74,m75_79,m80_84,m85+,ind
520,"York-Hanover, PA Metro Area",310M100US49620,435490,214931,13670,14987,14853,9086,6095,3135,1693,7605,12526,12533,13734,16503,16934,17145,13524,6399,7638,3918,6177,5771,5042,3412,2551,49620
521,"Youngstown-Warren-Boardman, OH-PA Metro Area",310M100US49660,564804,274119,15300,15289,19825,12038,7936,3441,4240,8829,14842,15155,16034,17941,19826,22481,20188,6955,11674,5470,8153,9191,7339,7128,4844,49660
522,"Yuba City, CA Metro Area",310M100US49700,167194,83083,6468,6672,6349,4029,2007,1045,1346,3680,6321,6440,5668,4431,6157,5081,4896,1417,2185,1170,1760,2569,1621,1180,591,49700
523,"Yuma, AZ Metro Area",310M100US49740,196509,98585,7437,6770,8723,4981,3652,2058,1478,4018,6661,5891,4570,6661,5626,5896,4491,2042,2833,1826,2120,4285,3544,1545,1477,49740
524,"Zanesville, OH Micro Area",310M100US49780,86142,41854,2772,2923,2989,1827,1205,641,418,1541,2389,2423,2548,3160,2780,3267,3017,1072,1506,849,1345,868,850,1095,369,49780


In [219]:
male.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5203 entries, 0 to 517
Data columns (total 29 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   CBSA    5203 non-null   object
 1   GEOID   5203 non-null   object
 2   total   5203 non-null   object
 3   mtotal  5203 non-null   object
 4   mu5     5203 non-null   object
 5   m5_9    5203 non-null   object
 6   m10_14  5203 non-null   object
 7   m15_17  5203 non-null   object
 8   m18_19  5203 non-null   object
 9   m20     5203 non-null   object
 10  m21     5203 non-null   object
 11  m22_24  5203 non-null   object
 12  m25_29  5203 non-null   object
 13  m30_34  5203 non-null   object
 14  m35_39  5203 non-null   object
 15  m40_44  5203 non-null   object
 16  m45_49  5203 non-null   object
 17  m50_54  5203 non-null   object
 18  m55_59  5203 non-null   object
 19  m60_61  5203 non-null   object
 20  m62_64  5203 non-null   object
 21  m65_66  5203 non-null   object
 22  m67_69  5203 non-null   o

In [226]:
geo = male.CBSA.unique()
print(geo.size)

686


In [234]:
#index into where the Nashville MSA is to check the rankings out
twentynineteen = male.loc[male['year'] == '2019'].reset_index(drop = True)

In [239]:
twentynineteen.head()

,CBSA,GEOID,total,mtotal,mu5,m5_9,m10_14,m15_17,m18_19,m20,m21,m22_24,m25_29,m30_34,m35_39,m40_44,m45_49,m50_54,m55_59,m60_61,m62_64,m65_66,m67_69,m70_74,m75_79,m80_84,m85+,ind,year
0,"Aberdeen, WA Micro Area",310M500US10140,75061,37879,1619,1749,2900,1006,278,659,402,1075,2213,2554,1767,2351,2576,2837,3118,983,1840,1030,1433,2606,1488,779,616,10140,2019
1,"Abilene, TX Metro Area",310M500US10180,171795,86747,5846,5253,6290,3630,2717,1861,1482,5169,7001,6303,5812,6603,3902,4210,5078,2140,2063,1859,2328,2317,2244,1219,1420,10180,2019
2,"Adrian, MI Micro Area",310M500US10300,98451,49744,2675,2573,2947,2112,1262,1009,572,1908,2633,3076,3292,3107,3254,3232,3804,1453,2012,756,2249,2513,1991,715,599,10300,2019
3,"Aguadilla-Isabela, PR Metro Area",310M500US10380,288877,141571,5475,6916,8129,5601,3506,2835,2129,5892,9027,7950,9616,9182,8609,8155,10266,5002,4447,3254,4804,8572,6043,4682,1479,10380,2019
4,"Akron, OH Metro Area",310M500US10420,703479,343036,18926,20794,20442,12747,10160,4423,4617,16881,24391,21529,19632,20457,21008,21930,24915,9575,14369,8527,10365,15810,9115,6956,5467,10420,2019


In [240]:
twentynineteen.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 518 entries, 0 to 517
Data columns (total 29 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   CBSA    518 non-null    object
 1   GEOID   518 non-null    object
 2   total   518 non-null    object
 3   mtotal  518 non-null    object
 4   mu5     518 non-null    object
 5   m5_9    518 non-null    object
 6   m10_14  518 non-null    object
 7   m15_17  518 non-null    object
 8   m18_19  518 non-null    object
 9   m20     518 non-null    object
 10  m21     518 non-null    object
 11  m22_24  518 non-null    object
 12  m25_29  518 non-null    object
 13  m30_34  518 non-null    object
 14  m35_39  518 non-null    object
 15  m40_44  518 non-null    object
 16  m45_49  518 non-null    object
 17  m50_54  518 non-null    object
 18  m55_59  518 non-null    object
 19  m60_61  518 non-null    object
 20  m62_64  518 non-null    object
 21  m65_66  518 non-null    object
 22  m67_69  518 non-null    ob

In [238]:
male2019.head()

,CBSA,GEOID,total,mtotal,mu5,m5_9,m10_14,m15_17,m18_19,m20,m21,m22_24,m25_29,m30_34,m35_39,m40_44,m45_49,m50_54,m55_59,m60_61,m62_64,m65_66,m67_69,m70_74,m75_79,m80_84,m85+,ind
0,"Aberdeen, WA Micro Area",310M500US10140,75061,37879,1619,1749,2900,1006,278,659,402,1075,2213,2554,1767,2351,2576,2837,3118,983,1840,1030,1433,2606,1488,779,616,10140
1,"Abilene, TX Metro Area",310M500US10180,171795,86747,5846,5253,6290,3630,2717,1861,1482,5169,7001,6303,5812,6603,3902,4210,5078,2140,2063,1859,2328,2317,2244,1219,1420,10180
2,"Adrian, MI Micro Area",310M500US10300,98451,49744,2675,2573,2947,2112,1262,1009,572,1908,2633,3076,3292,3107,3254,3232,3804,1453,2012,756,2249,2513,1991,715,599,10300
3,"Aguadilla-Isabela, PR Metro Area",310M500US10380,288877,141571,5475,6916,8129,5601,3506,2835,2129,5892,9027,7950,9616,9182,8609,8155,10266,5002,4447,3254,4804,8572,6043,4682,1479,10380
4,"Akron, OH Metro Area",310M500US10420,703479,343036,18926,20794,20442,12747,10160,4423,4617,16881,24391,21529,19632,20457,21008,21930,24915,9575,14369,8527,10365,15810,9115,6956,5467,10420


In [241]:
male2019.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 518 entries, 0 to 517
Data columns (total 28 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   CBSA    518 non-null    object
 1   GEOID   518 non-null    object
 2   total   518 non-null    object
 3   mtotal  518 non-null    object
 4   mu5     518 non-null    object
 5   m5_9    518 non-null    object
 6   m10_14  518 non-null    object
 7   m15_17  518 non-null    object
 8   m18_19  518 non-null    object
 9   m20     518 non-null    object
 10  m21     518 non-null    object
 11  m22_24  518 non-null    object
 12  m25_29  518 non-null    object
 13  m30_34  518 non-null    object
 14  m35_39  518 non-null    object
 15  m40_44  518 non-null    object
 16  m45_49  518 non-null    object
 17  m50_54  518 non-null    object
 18  m55_59  518 non-null    object
 19  m60_61  518 non-null    object
 20  m62_64  518 non-null    object
 21  m65_66  518 non-null    object
 22  m67_69  518 non-null    ob

In [235]:
#male 2010 and total
api_key = '24fc7d81b74510d599f702dbd408fb18e1466d81'
url_str= 'https://api.census.gov/data/2010/acs/acs1?get=NAME,GEO_ID,B01001_001E,B01001_002E,B01001_003E,B01001_004E,B01001_005E,B01001_006E,B01001_007E,B01001_008E,B01001_009E,B01001_010E,B01001_011E,B01001_012E,B01001_013E,B01001_014E,B01001_015E,B01001_016E,B01001_017E,B01001_018E,B01001_019E,B01001_020E,B01001_021E,B01001_022E,B01001_023E,B01001_024E,B01001_025E&for=metropolitan%20statistical%20area/micropolitan%20statistical%20area:*&key='+api_key

predicates= {}
get_vars= ["NAME","GEO_ID",'B01001_001E','B01001_020E','B01001_003E','B01001_004E','B01001_005E','B01001_006E','B01001_007E','B01001_008E','B01001_009E','B01001_010E','B01001_011E','B01001_012E','B01001_013E','B01001_014E','B01001_015E','B01001_016E','B01001_017E','B01001_018E','B01001_019E','B01001_020E','B01001_021E','B01001_022E','B01001_023E','B01001_024E','B01001_025E']
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:*"
male2010= requests.get(url_str, params= predicates)

col_names = ['CBSA','GEOID','total','mtotal','mu5','m5_9','m10_14','m15_17','m18_19','m20','m21','m22_24','m25_29','m30_34',
            'm35_39','m40_44','m45_49','m50_54','m55_59','m60_61','m62_64','m65_66','m67_69','m70_74','m75_79','m80_84','m85+','ind']
male2010=pd.DataFrame(columns=col_names, data=male2010.json()[1:])

In [237]:
#male 2019 and total
api_key = '24fc7d81b74510d599f702dbd408fb18e1466d81'
url_str= 'https://api.census.gov/data/2019/acs/acs1?get=NAME,GEO_ID,B01001_001E,B01001_002E,B01001_003E,B01001_004E,B01001_005E,B01001_006E,B01001_007E,B01001_008E,B01001_009E,B01001_010E,B01001_011E,B01001_012E,B01001_013E,B01001_014E,B01001_015E,B01001_016E,B01001_017E,B01001_018E,B01001_019E,B01001_020E,B01001_021E,B01001_022E,B01001_023E,B01001_024E,B01001_025E&for=metropolitan%20statistical%20area/micropolitan%20statistical%20area:*&key='+api_key

predicates= {}
get_vars= ["NAME","GEO_ID",'B01001_001E','B01001_020E','B01001_003E','B01001_004E','B01001_005E','B01001_006E','B01001_007E','B01001_008E','B01001_009E','B01001_010E','B01001_011E','B01001_012E','B01001_013E','B01001_014E','B01001_015E','B01001_016E','B01001_017E','B01001_018E','B01001_019E','B01001_020E','B01001_021E','B01001_022E','B01001_023E','B01001_024E','B01001_025E']
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:*"
male2019= requests.get(url_str, params= predicates)

col_names = ['CBSA','GEOID','total','mtotal','mu5','m5_9','m10_14','m15_17','m18_19','m20','m21','m22_24','m25_29','m30_34',
            'm35_39','m40_44','m45_49','m50_54','m55_59','m60_61','m62_64','m65_66','m67_69','m70_74','m75_79','m80_84','m85+','ind']
male2019=pd.DataFrame(columns=col_names, data=male2019.json()[1:])

In [206]:
# #male 2014 and total
# api_key = '24fc7d81b74510d599f702dbd408fb18e1466d81'
# url_str= 'https://api.census.gov/data/2014/acs/acs1?get=NAME,GEO_ID,B01001_001E,B01001_002E,B01001_003E,B01001_004E,B01001_005E,B01001_006E,B01001_007E,B01001_008E,B01001_009E,B01001_010E,B01001_011E,B01001_012E,B01001_013E,B01001_014E,B01001_015E,B01001_016E,B01001_017E,B01001_018E,B01001_019E,B01001_020E,B01001_021E,B01001_022E,B01001_023E,B01001_024E,B01001_025E&for=metropolitan%20statistical%20area/micropolitan%20statistical%20area:*&key='+api_key

# predicates= {}
# get_vars= ["NAME","GEO_ID",'B01001_001E','B01001_020E','B01001_003E','B01001_004E','B01001_005E','B01001_006E','B01001_007E','B01001_008E','B01001_009E','B01001_010E','B01001_011E','B01001_012E','B01001_013E','B01001_014E','B01001_015E','B01001_016E','B01001_017E','B01001_018E','B01001_019E','B01001_020E','B01001_021E','B01001_022E','B01001_023E','B01001_024E','B01001_025E']
# predicates["get"]= ",". join(get_vars)
# predicates["for"]= "state:*"
# male2014= requests.get(url_str, params= predicates)

# col_names = ['CBSA','GEOID','total','mtotal','mu5','m5_9','m10_14','m15_17','m18_19','m20','m21','m22_24','m25_29','m30_34',
#             'm35_39','m40_44','m45_49','m50_54','m55_59','m60_61','m62_64','m65_66','m67_69','m70_74','m75_79','m80_84','m85+','ind']
# male2014=pd.DataFrame(columns=col_names, data=male2014.json()[1:])

In [207]:
# #male 2015 and total
# api_key = '24fc7d81b74510d599f702dbd408fb18e1466d81'
# url_str= 'https://api.census.gov/data/2015/acs/acs1?get=NAME,GEO_ID,B01001_001E,B01001_002E,B01001_003E,B01001_004E,B01001_005E,B01001_006E,B01001_007E,B01001_008E,B01001_009E,B01001_010E,B01001_011E,B01001_012E,B01001_013E,B01001_014E,B01001_015E,B01001_016E,B01001_017E,B01001_018E,B01001_019E,B01001_020E,B01001_021E,B01001_022E,B01001_023E,B01001_024E,B01001_025E&for=metropolitan%20statistical%20area/micropolitan%20statistical%20area:*&key='+api_key

# predicates= {}
# get_vars= ["NAME","GEO_ID",'B01001_001E','B01001_020E','B01001_003E','B01001_004E','B01001_005E','B01001_006E','B01001_007E','B01001_008E','B01001_009E','B01001_010E','B01001_011E','B01001_012E','B01001_013E','B01001_014E','B01001_015E','B01001_016E','B01001_017E','B01001_018E','B01001_019E','B01001_020E','B01001_021E','B01001_022E','B01001_023E','B01001_024E','B01001_025E']
# predicates["get"]= ",". join(get_vars)
# predicates["for"]= "state:*"
# male2015= requests.get(url_str, params= predicates)

# col_names = ['CBSA','GEOID','total','mtotal','mu5','m5_9','m10_14','m15_17','m18_19','m20','m21','m22_24','m25_29','m30_34',
#             'm35_39','m40_44','m45_49','m50_54','m55_59','m60_61','m62_64','m65_66','m67_69','m70_74','m75_79','m80_84','m85+','ind']
# male2015=pd.DataFrame(columns=col_names, data=male2015.json()[1:])

In [208]:
# #male 2016 and total
# api_key = '24fc7d81b74510d599f702dbd408fb18e1466d81'
# url_str= 'https://api.census.gov/data/2016/acs/acs1?get=NAME,GEO_ID,B01001_001E,B01001_002E,B01001_003E,B01001_004E,B01001_005E,B01001_006E,B01001_007E,B01001_008E,B01001_009E,B01001_010E,B01001_011E,B01001_012E,B01001_013E,B01001_014E,B01001_015E,B01001_016E,B01001_017E,B01001_018E,B01001_019E,B01001_020E,B01001_021E,B01001_022E,B01001_023E,B01001_024E,B01001_025E&for=metropolitan%20statistical%20area/micropolitan%20statistical%20area:*&key='+api_key

# predicates= {}
# get_vars= ["NAME","GEO_ID",'B01001_001E','B01001_020E','B01001_003E','B01001_004E','B01001_005E','B01001_006E','B01001_007E','B01001_008E','B01001_009E','B01001_010E','B01001_011E','B01001_012E','B01001_013E','B01001_014E','B01001_015E','B01001_016E','B01001_017E','B01001_018E','B01001_019E','B01001_020E','B01001_021E','B01001_022E','B01001_023E','B01001_024E','B01001_025E']
# predicates["get"]= ",". join(get_vars)
# predicates["for"]= "state:*"
# male2016= requests.get(url_str, params= predicates)

# col_names = ['CBSA','GEOID','total','mtotal','mu5','m5_9','m10_14','m15_17','m18_19','m20','m21','m22_24','m25_29','m30_34',
#             'm35_39','m40_44','m45_49','m50_54','m55_59','m60_61','m62_64','m65_66','m67_69','m70_74','m75_79','m80_84','m85+','ind']
# male2016=pd.DataFrame(columns=col_names, data=male2016.json()[1:])

In [209]:
# #male 2017 and total
# api_key = '24fc7d81b74510d599f702dbd408fb18e1466d81'
# url_str= 'https://api.census.gov/data/2017/acs/acs1?get=NAME,GEO_ID,B01001_001E,B01001_002E,B01001_003E,B01001_004E,B01001_005E,B01001_006E,B01001_007E,B01001_008E,B01001_009E,B01001_010E,B01001_011E,B01001_012E,B01001_013E,B01001_014E,B01001_015E,B01001_016E,B01001_017E,B01001_018E,B01001_019E,B01001_020E,B01001_021E,B01001_022E,B01001_023E,B01001_024E,B01001_025E&for=metropolitan%20statistical%20area/micropolitan%20statistical%20area:*&key='+api_key

# predicates= {}
# get_vars= ["NAME","GEO_ID",'B01001_001E','B01001_020E','B01001_003E','B01001_004E','B01001_005E','B01001_006E','B01001_007E','B01001_008E','B01001_009E','B01001_010E','B01001_011E','B01001_012E','B01001_013E','B01001_014E','B01001_015E','B01001_016E','B01001_017E','B01001_018E','B01001_019E','B01001_020E','B01001_021E','B01001_022E','B01001_023E','B01001_024E','B01001_025E']
# predicates["get"]= ",". join(get_vars)
# predicates["for"]= "state:*"
# male2017= requests.get(url_str, params= predicates)

# col_names = ['CBSA','GEOID','total','mtotal','mu5','m5_9','m10_14','m15_17','m18_19','m20','m21','m22_24','m25_29','m30_34',
#             'm35_39','m40_44','m45_49','m50_54','m55_59','m60_61','m62_64','m65_66','m67_69','m70_74','m75_79','m80_84','m85+','ind']
# male2017=pd.DataFrame(columns=col_names, data=male2017.json()[1:])

In [210]:
# #male 2018 and total
# api_key = '24fc7d81b74510d599f702dbd408fb18e1466d81'
# url_str= 'https://api.census.gov/data/2018/acs/acs1?get=NAME,GEO_ID,B01001_001E,B01001_002E,B01001_003E,B01001_004E,B01001_005E,B01001_006E,B01001_007E,B01001_008E,B01001_009E,B01001_010E,B01001_011E,B01001_012E,B01001_013E,B01001_014E,B01001_015E,B01001_016E,B01001_017E,B01001_018E,B01001_019E,B01001_020E,B01001_021E,B01001_022E,B01001_023E,B01001_024E,B01001_025E&for=metropolitan%20statistical%20area/micropolitan%20statistical%20area:*&key='+api_key

# predicates= {}
# get_vars= ["NAME","GEO_ID",'B01001_001E','B01001_020E','B01001_003E','B01001_004E','B01001_005E','B01001_006E','B01001_007E','B01001_008E','B01001_009E','B01001_010E','B01001_011E','B01001_012E','B01001_013E','B01001_014E','B01001_015E','B01001_016E','B01001_017E','B01001_018E','B01001_019E','B01001_020E','B01001_021E','B01001_022E','B01001_023E','B01001_024E','B01001_025E']
# predicates["get"]= ",". join(get_vars)
# predicates["for"]= "state:*"
# male2018= requests.get(url_str, params= predicates)

# col_names = ['CBSA','GEOID','total','mtotal','mu5','m5_9','m10_14','m15_17','m18_19','m20','m21','m22_24','m25_29','m30_34',
#             'm35_39','m40_44','m45_49','m50_54','m55_59','m60_61','m62_64','m65_66','m67_69','m70_74','m75_79','m80_84','m85+','ind']
# male2018=pd.DataFrame(columns=col_names, data=male2018.json()[1:])

In [211]:
# #male 2019 and total
# api_key = '24fc7d81b74510d599f702dbd408fb18e1466d81'
# url_str= 'https://api.census.gov/data/2019/acs/acs1?get=NAME,GEO_ID,B01001_001E,B01001_002E,B01001_003E,B01001_004E,B01001_005E,B01001_006E,B01001_007E,B01001_008E,B01001_009E,B01001_010E,B01001_011E,B01001_012E,B01001_013E,B01001_014E,B01001_015E,B01001_016E,B01001_017E,B01001_018E,B01001_019E,B01001_020E,B01001_021E,B01001_022E,B01001_023E,B01001_024E,B01001_025E&for=metropolitan%20statistical%20area/micropolitan%20statistical%20area:*&key='+api_key

# predicates= {}
# get_vars= ["NAME","GEO_ID",'B01001_001E','B01001_020E','B01001_003E','B01001_004E','B01001_005E','B01001_006E','B01001_007E','B01001_008E','B01001_009E','B01001_010E','B01001_011E','B01001_012E','B01001_013E','B01001_014E','B01001_015E','B01001_016E','B01001_017E','B01001_018E','B01001_019E','B01001_020E','B01001_021E','B01001_022E','B01001_023E','B01001_024E','B01001_025E']
# predicates["get"]= ",". join(get_vars)
# predicates["for"]= "state:*"
# male2019= requests.get(url_str, params= predicates)

# col_names = ['CBSA','GEOID','total','mtotal','mu5','m5_9','m10_14','m15_17','m18_19','m20','m21','m22_24','m25_29','m30_34',
#             'm35_39','m40_44','m45_49','m50_54','m55_59','m60_61','m62_64','m65_66','m67_69','m70_74','m75_79','m80_84','m85+','ind']
# male2019=pd.DataFrame(columns=col_names, data=male2019.json()[1:])

In [212]:
# #female old
# api_key = '24fc7d81b74510d599f702dbd408fb18e1466d81'
# url_str= 'https://api.census.gov/data/2014/acs/acs5?get=NAME,GEO_ID,B01001_026E,B01001_027E,B01001_028E,B01001_029E,B01001_030E,B01001_031E,B01001_032E,B01001_033E,B01001_034E,B01001_035E,B01001_036E,B01001_037E,B01001_038E,B01001_039E,B01001_040E,B01001_041E,B01001_042E,B01001_043E,B01001_044E,B01001_045E,B01001_046E,B01001_047E,B01001_048E,B01001_049E&for=metropolitan%20statistical%20area/micropolitan%20statistical%20area:*&key='+api_key

# predicates= {}
# get_vars= ["NAME","GEO_ID",'B01001_026E','B01001_027E','B01001_028E','B01001_029E','B01001_030E','B01001_031E','B01001_032E','B01001_033E','B01001_034E','B01001_035E','B01001_036E','B01001_037E','B01001_038E','B01001_039E','B01001_040E','B01001_041E','B01001_042E','B01001_043E','B01001_044E','B01001_045E','B01001_046E','B01001_047E','B01001_048E','B01001_049E']
# predicates["get"]= ",". join(get_vars)
# predicates["for"]= "state:*"
# femaleold= requests.get(url_str, params= predicates)

# col_names = ['CBSA','GEOID','ftotal', 'fu5','f5_9','f10_14','f15_17','f18_19','f20','f21','f22_24','f25_29','f30_34',
#             'f35_39','f40_44','f45_49','f50_54','f55_59','f60_61','f62_64','f65_66','f67_69','f70_74','f75_79','f80_84','f85+','ind']
# femaleold=pd.DataFrame(columns=col_names, data=femaleold.json()[1:])

In [213]:
# #home value old
# api_key = '24fc7d81b74510d599f702dbd408fb18e1466d81'
# url_str= 'https://api.census.gov/data/2014/acs/acs5?get=NAME,GEO_ID,B25077_001E&for=metropolitan%20statistical%20area/micropolitan%20statistical%20area:*&key='+api_key

# predicates= {}
# get_vars= ["NAME","GEO_ID",'B25077_001E']
# predicates["get"]= ",". join(get_vars)
# predicates["for"]= "state:*"
# homeold= requests.get(url_str, params= predicates)

# col_names = ['CBSA','GEOID','homevalue','ind']
# homeold=pd.DataFrame(columns=col_names, data=homeold.json()[1:])

In [222]:
#for loop for renaming them with an O* for old at the beginning of each column
for col in old.columns:
    old.rename(columns = {col:'O*'+col}, inplace = True)

In [223]:
old.head()

,O*CBSA_x,O*GEOID,O*total,O*mtotal,O*mu5,O*m5_9,O*m10_14,O*m15_17,O*m18_19,O*m20,O*m21,O*m22_24,O*m25_29,O*m30_34,O*m35_39,O*m40_44,O*m45_49,O*m50_54,O*m55_59,O*m60_61,O*m62_64,O*m65_66,O*m67_69,O*m70_74,O*m75_79,O*m80_84,O*m85+,O*ind_x,O*CBSA_y,O*ftotal,O*fu5,O*f5_9,O*f10_14,O*f15_17,O*f18_19,O*f20,O*f21,O*f22_24,O*f25_29,O*f30_34,O*f35_39,O*f40_44,O*f45_49,O*f50_54,O*f55_59,O*f60_61,O*f62_64,O*f65_66,O*f67_69,O*f70_74,O*f75_79,O*f80_84,O*f85+,O*ind_y,O*CBSA,O*homevalue,O*ind
0,"Homosassa Springs, FL Metro Area",310M200US26140,139771,67497,2639,2404,3599,2170,1310,544,713,1735,2686,2332,3038,2785,4035,4587,4835,2170,3453,2319,4543,5434,4898,3099,2169,26140,"Homosassa Springs, FL Metro Area",72274,2793,2634,3197,2052,1241,587,719,1283,2513,2446,2769,3531,4279,5174,5917,2541,4107,2612,4327,6294,4520,3354,3384,26140,"Homosassa Springs, FL Metro Area",113500,26140
1,"Hickory-Lenoir-Morganton, NC Metro Area",310M200US25860,363936,180006,10129,11631,11662,7909,5182,2955,2286,6349,9750,10134,11868,12959,13986,13513,12607,4571,6761,4305,5277,6744,4547,2959,1922,25860,"Hickory-Lenoir-Morganton, NC Metro Area",183930,9973,10596,12065,6895,4132,2428,2220,5832,9471,10089,11181,13228,13332,13845,12833,5074,7537,4340,6002,7976,6162,4219,4500,25860,"Hickory-Lenoir-Morganton, NC Metro Area",120200,25860
2,"Hobbs, NM Micro Area",310M200US26020,66876,34219,2909,2708,2964,1522,1137,500,440,1518,2612,2675,2122,2092,2090,2206,1854,696,868,456,626,806,675,448,295,26020,"Hobbs, NM Micro Area",32657,2881,2747,2692,1509,892,648,378,1275,2283,2113,2205,1710,1907,2109,2067,583,803,527,625,898,833,580,392,26020,"Hobbs, NM Micro Area",101400,26020
3,"Holland, MI Micro Area",310M200US26090,112266,55910,3599,4145,4271,2617,1315,736,512,1850,3009,3209,3366,3708,4225,4500,4298,1415,1887,1044,1437,1980,1224,935,628,26090,"Holland, MI Micro Area",56356,3468,3958,3982,2504,1289,654,701,1655,2959,3284,3277,3729,4136,4547,3942,1721,1965,1112,1695,1934,1579,946,1319,26090,"Holland, MI Micro Area",137500,26090
4,"Hood River, OR Micro Area",310M200US26220,22620,11169,800,819,850,381,229,89,146,483,694,732,535,958,818,856,747,305,368,197,191,459,233,157,122,26220,"Hood River, OR Micro Area",11451,716,742,860,574,154,45,140,511,631,681,824,768,800,866,819,263,326,214,234,419,322,226,316,26220,"Hood River, OR Micro Area",309500,26220


In [224]:
#rename the O*GEOID to the reverted column name to join on it
old = old.rename(columns = {'O*GEOID':'GEOID', 'O*CBSA':'CBSA'})

In [225]:
#make sure the datatypes are correct
new['GEOID'] = new['GEOID'].astype(str)
old['GEOID'] = old['GEOID'].astype(str)
new['CBSA'] = new['CBSA'].astype(str)
old['CBSA'] = old['CBSA'].astype(str)

In [226]:
data = pd.merge(new, old, on='CBSA', how = 'inner')

Check out our datatypes and any extra columns from the join that are no longer necessary.

In [227]:
#set parameter to show all columns
pd.set_option('max_columns', None)

In [228]:
data.head()

,CBSA_x,GEOID_x,total,mtotal,mu5,m5_9,m10_14,m15_17,m18_19,m20,m21,m22_24,m25_29,m30_34,m35_39,m40_44,m45_49,m50_54,m55_59,m60_61,m62_64,m65_66,m67_69,m70_74,m75_79,m80_84,m85+,ind_x,CBSA_y,ftotal,fu5,f5_9,f10_14,f15_17,f18_19,f20,f21,f22_24,f25_29,f30_34,f35_39,f40_44,f45_49,f50_54,f55_59,f60_61,f62_64,f65_66,f67_69,f70_74,f75_79,f80_84,f85+,ind_y,CBSA,homevalue,ind,O*CBSA_x,GEOID_y,O*total,O*mtotal,O*mu5,O*m5_9,O*m10_14,O*m15_17,O*m18_19,O*m20,O*m21,O*m22_24,O*m25_29,O*m30_34,O*m35_39,O*m40_44,O*m45_49,O*m50_54,O*m55_59,O*m60_61,O*m62_64,O*m65_66,O*m67_69,O*m70_74,O*m75_79,O*m80_84,O*m85+,O*ind_x,O*CBSA_y,O*ftotal,O*fu5,O*f5_9,O*f10_14,O*f15_17,O*f18_19,O*f20,O*f21,O*f22_24,O*f25_29,O*f30_34,O*f35_39,O*f40_44,O*f45_49,O*f50_54,O*f55_59,O*f60_61,O*f62_64,O*f65_66,O*f67_69,O*f70_74,O*f75_79,O*f80_84,O*f85+,O*ind_y,O*homevalue,O*ind
0,"Big Stone Gap, VA Micro Area",310M500US13720,42456,22025,1160,1149,1253,776,583,356,238,1035,1791,1490,1432,1480,1430,1388,1465,768,853,425,861,858,559,340,335,13720,"Big Stone Gap, VA Micro Area",20431,1054,1297,1012,702,447,345,172,606,1360,1143,1272,1123,1317,1248,1611,596,1047,740,646,947,638,573,535,13720,"Big Stone Gap, VA Micro Area",86200,13720,"Big Stone Gap, VA Micro Area",310M200US13720,60499,30657,1614,1942,1599,1070,826,443,333,1527,2149,2257,1918,2250,2176,2222,1710,959,1513,668,951,1034,755,430,311,13720,"Big Stone Gap, VA Micro Area",29842,1605,1674,1795,1076,801,521,484,1008,1650,1667,1911,1888,2065,2158,1961,834,1569,605,860,1347,1013,646,704,13720,82800,13720
1,"Billings, MT Metro Area",310M500US13740,179071,88433,5510,6170,5862,3575,1936,774,904,3687,5856,5880,5701,5403,5355,5446,6014,2780,3459,2176,3299,3299,2395,1420,1532,13740,"Billings, MT Metro Area",90638,5301,5588,6219,3210,1857,743,944,3251,5905,6083,5596,5425,5204,5681,6679,2569,3502,2014,3357,4053,2710,2050,2697,13740,"Billings, MT Metro Area",227900,13740,"Billings, MT Metro Area",310M200US13740,162892,79941,5415,5422,5184,3152,2033,993,1071,2735,5529,5384,4883,4960,5268,5976,5848,2202,2950,1611,2004,2704,1942,1586,1089,13740,"Billings, MT Metro Area",82951,5205,5494,5052,3050,1722,1018,922,3161,5581,5401,4971,4769,5403,6224,5813,2420,3039,1952,2190,2776,2276,1989,2523,13740,187100,13740
2,"Binghamton, NY Metro Area",310M500US13780,241874,119221,6476,7128,6619,4518,4705,2593,2894,5923,7021,6628,6571,5879,6906,7872,9256,3942,4230,2808,3735,5025,4163,2167,2162,13780,"Binghamton, NY Metro Area",122653,6104,6196,6595,4263,4564,2517,2145,5874,6639,6328,6482,6330,6850,8269,9712,3461,4673,3223,3893,5733,4678,3388,4736,13780,"Binghamton, NY Metro Area",117600,13780,"Binghamton, NY Metro Area",310M200US13780,249261,122423,6602,7244,7375,4749,5299,2628,3001,5257,7325,6938,6854,6644,8153,9796,8886,3167,4452,2357,3266,4393,3640,2605,1792,13780,"Binghamton, NY Metro Area",126838,6273,6931,6989,4635,4683,2297,2124,5849,7032,6580,6308,7209,8776,10084,8672,3107,5202,2629,3967,4790,4246,3734,4721,13780,108800,13780
3,"Birmingham-Hoover, AL Metro Area",310M500US13820,1085330,521496,34020,35330,36232,22069,12928,6687,6550,19721,37169,35364,34445,33803,34093,33920,34366,13750,18921,11914,15273,18629,11772,8176,6364,13820,"Birmingham-Hoover, AL Metro Area",563834,33618,34098,34317,20814,12937,7540,5998,19872,38767,37822,36442,36075,36622,36524,38226,15788,21722,13729,17591,23233,16738,12956,12405,13820,"Birmingham-Hoover, AL Metro Area",165200,13820,"Birmingham-Hoover, AL Metro Area",310M200US13820,1135534,546799,37407,37502,38646,23132,14436,7607,7440,20762,37301,37933,34997,38338,37785,39511,36556,13426,18552,10763,12759,16487,11792,7820,5847,13820,"Birmingham-Hoover, AL Metro Area",588735,35890,35840,37332,21865,13503,6454,7941,22869,40099,39780,36947,39815,40045,42833,40701,15300,21088,12123,15742,20136,16587,13124,12721,13820,144000,13820
4,"Bismarck, ND Metro Area",310M500US13900,127503,64036,4382,4386,4002,2228,1558,731,865,2573,4842,5133,4758,3710,3722,4016,4350,1672,2137,1356,1865,2200,1477,940,1133,13900,"Bismarck, ND Metro Ar

Drop the indices and extra CBSA name column, rename the one you're going to keep, then convert all numeric columns to int64 as we're going to be performing mathematical operations on them.

In [229]:
#drop the indices and extra state name
data = data.drop(columns=['ind','ind_x','CBSA_y','ind_y','CBSA','ind','O*CBSA_x','GEOID_y','O*ind_x','O*CBSA_y','O*ind_y','O*ind','GEOID_y'])

In [230]:
data = data.rename(columns={'CBSA_x':'CBSA','GEOID_x':'GEOID'})

In [231]:
#convert all numeric columns to integers
cols = ['total','mtotal','mu5','m5_9','m10_14','m15_17','m18_19','m20','m21','m22_24','m25_29','m30_34',
        'm35_39','m40_44','m45_49','m50_54','m55_59','m60_61','m62_64','m65_66','m67_69','m70_74','m75_79',
        'm80_84','m85+','ftotal', 'fu5','f5_9','f10_14','f15_17','f18_19','f20','f21','f22_24','f25_29',
        'f30_34','f35_39','f40_44','f45_49','f50_54','f55_59','f60_61','f62_64','f65_66','f67_69','f70_74',
        'f75_79','f80_84','f85+','homevalue','O*total','O*mtotal','O*mu5','O*m5_9','O*m10_14','O*m15_17','O*m18_19','O*m20',
        'O*m21','O*m22_24','O*m25_29','O*m30_34','O*m35_39','O*m40_44','O*m45_49','O*m50_54','O*m55_59','O*m60_61','O*m62_64','O*m65_66',
        'O*m67_69','O*m70_74','O*m75_79','O*m80_84','O*m85+','O*ftotal', 'O*fu5','O*f5_9','O*f10_14','O*f15_17','O*f18_19','O*f20','O*f21',
        'O*f22_24','O*f25_29','O*f30_34','O*f35_39','O*f40_44','O*f45_49','O*f50_54','O*f55_59','O*f60_61','O*f62_64','O*f65_66','O*f67_69',
        'O*f70_74','O*f75_79','O*f80_84','O*f85+','O*homevalue']

data[cols]=data[cols].apply(pd.to_numeric, errors='coerce', axis=1)

Check that the last few steps were effective.

In [232]:
data.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 859 entries, 0 to 858
Data columns (total 102 columns):
 #    Column       Dtype 
---   ------       ----- 
 0    CBSA         object
 1    GEOID        object
 2    total        int64 
 3    mtotal       int64 
 4    mu5          int64 
 5    m5_9         int64 
 6    m10_14       int64 
 7    m15_17       int64 
 8    m18_19       int64 
 9    m20          int64 
 10   m21          int64 
 11   m22_24       int64 
 12   m25_29       int64 
 13   m30_34       int64 
 14   m35_39       int64 
 15   m40_44       int64 
 16   m45_49       int64 
 17   m50_54       int64 
 18   m55_59       int64 
 19   m60_61       int64 
 20   m62_64       int64 
 21   m65_66       int64 
 22   m67_69       int64 
 23   m70_74       int64 
 24   m75_79       int64 
 25   m80_84       int64 
 26   m85+         int64 
 27   ftotal       int64 
 28   fu5          int64 
 29   f5_9         int64 
 30   f10_14       int64 
 31   f15_17       int64 
 32   f18_

Create columns of totals for each one of these age groups.

In [236]:
#create totals columns for "new":
data['tu5'] = data['mu5']+data['fu5']
data['t5_9'] = data['m5_9']+data['f5_9']
data['t10_14'] = data['m10_14']+data['f10_14']
data['t15_17'] = data['m15_17']+data['f15_17']
data['t18_19'] = data['m18_19']+data['f18_19']
data['t20'] = data['m20']+data['f20']
data['t21'] = data['m21']+data['f21']
data['t22_24'] = data['m22_24']+data['f22_24']
data['t25_29'] = data['m25_29']+data['f25_29']
data['t30_34'] = data['m30_34']+data['f30_34']
data['t35_39'] = data['m35_39']+data['f35_39']
data['t40_44'] = data['m40_44']+data['f40_44']
data['t45_49'] = data['m45_49']+data['f45_49']
data['t50_54'] = data['m50_54']+data['f50_54']
data['t55_59'] = data['m55_59']+data['f55_59']
data['t60_61'] = data['m60_61']+data['f60_61']
data['t62_64'] = data['m62_64']+data['f62_64']
data['t65_66'] = data['m65_66']+data['f65_66']
data['t67_69'] = data['m67_69']+data['f67_69']
data['t70_74'] = data['m70_74']+data['f70_74']
data['t75_79'] = data['m75_79']+data['f75_79']
data['t80_84'] = data['m80_84']+data['f80_84']
data['t85+'] = data['m85+']+data['f85+']

data = data.copy()

In [237]:
#do the same for "old":
data['O*tu5'] = data['O*mu5']+data['O*fu5']
data['O*t5_9'] = data['O*m5_9']+data['O*f5_9']
data['O*t10_14'] = data['O*m10_14']+data['O*f10_14']
data['O*t15_17'] = data['O*m15_17']+data['O*f15_17']
data['O*t18_19'] = data['O*m18_19']+data['O*f18_19']
data['O*t20'] = data['O*m20']+data['O*f20']
data['O*t21'] = data['O*m21']+data['O*f21']
data['O*t22_24'] = data['O*m22_24']+data['O*f22_24']
data['O*t25_29'] = data['O*m25_29']+data['O*f25_29']
data['O*t30_34'] = data['O*m30_34']+data['O*f30_34']
data['O*t35_39'] = data['O*m35_39']+data['O*f35_39']
data['O*t40_44'] = data['O*m40_44']+data['O*f40_44']
data['O*t45_49'] = data['O*m45_49']+data['O*f45_49']
data['O*t50_54'] = data['O*m50_54']+data['O*f50_54']
data['O*t55_59'] = data['O*m55_59']+data['O*f55_59']
data['O*t60_61'] = data['O*m60_61']+data['O*f60_61']
data['O*t62_64'] = data['O*m62_64']+data['O*f62_64']
data['O*t65_66'] = data['O*m65_66']+data['O*f65_66']
data['O*t67_69'] = data['O*m67_69']+data['O*f67_69']
data['O*t70_74'] = data['O*m70_74']+data['O*f70_74']
data['O*t75_79'] = data['O*m75_79']+data['O*f75_79']
data['O*t80_84'] = data['O*m80_84']+data['O*f80_84']
data['O*t85+'] = data['O*m85+']+data['O*f85+']

data = data.copy()

## Create groups for analysis. Since we're looking at ageing services, it makes sense to group people into some rough "stage of life" that is predicated on services provided to them or their position in the workforce.

###### So we're going to do:  
- under 18  
- 18 - 64  
- 65+  

Note that we're also eliminating the gender divide, we can always come back and disaggregate this but it doesn't seem particularly relevant now.

In [239]:
#child
data['child'] = data['tu5']+data['t5_9']+data['t10_14']+data['t15_17']
data['O*child'] = data['O*tu5']+data['O*t5_9']+data['O*t10_14']+data['O*t15_17']
#taxbase
data['taxbase'] = data['t18_19']+data['t20']+data['t21']+data['t22_24']+data['t25_29']+data['t30_34']+data['t35_39']+data['t40_44']+data['t45_49']+data['t50_54']+data['t55_59']+data['t60_61']+data['t62_64']
data['O*taxbase'] = data['O*t18_19']+data['O*t20']+data['O*t21']+data['O*t22_24']+data['O*t25_29']+data['O*t30_34']+data['O*t35_39']+data['O*t40_44']+data['O*t45_49']+data['O*t50_54']+data['O*t55_59']+data['O*t60_61']+data['O*t62_64']
#over 65
data['O65'] = data['t65_66']+data['t67_69']+data['t70_74']+data['t75_79']+data['t80_84']+data['t85+']
data['O*O65'] = data['O*t65_66']+data['O*t67_69']+data['O*t70_74']+data['O*t75_79']+data['O*t80_84']+data['O*t85+']

data = data.copy()

Make these groups into shares of the total.

In [241]:
data['Pchild']=round(data['child']*100/data['total'],1)
data['Ptaxbase']=round(data['taxbase']*100/data['total'],1)
data['PO65']=round(data['O65']*100/data['total'],1)
data['O*Pchild']=round(data['O*child']*100/data['O*total'],1)
data['O*Ptaxbase']=round(data['O*taxbase']*100/data['O*total'],1)
data['O*PO65']=round(data['O*O65']*100/data['O*total'],1)

data = data.copy()

In [242]:
data.head(2)

,CBSA,GEOID,total,mtotal,mu5,m5_9,m10_14,m15_17,m18_19,m20,m21,m22_24,m25_29,m30_34,m35_39,m40_44,m45_49,m50_54,m55_59,m60_61,m62_64,m65_66,m67_69,m70_74,m75_79,m80_84,m85+,ftotal,fu5,f5_9,f10_14,f15_17,f18_19,f20,f21,f22_24,f25_29,f30_34,f35_39,f40_44,f45_49,f50_54,f55_59,f60_61,f62_64,f65_66,f67_69,f70_74,f75_79,f80_84,f85+,homevalue,O*total,O*mtotal,O*mu5,O*m5_9,O*m10_14,O*m15_17,O*m18_19,O*m20,O*m21,O*m22_24,O*m25_29,O*m30_34,O*m35_39,O*m40_44,O*m45_49,O*m50_54,O*m55_59,O*m60_61,O*m62_64,O*m65_66,O*m67_69,O*m70_74,O*m75_79,O*m80_84,O*m85+,O*ftotal,O*fu5,O*f5_9,O*f10_14,O*f15_17,O*f18_19,O*f20,O*f21,O*f22_24,O*f25_29,O*f30_34,O*f35_39,O*f40_44,O*f45_49,O*f50_54,O*f55_59,O*f60_61,O*f62_64,O*f65_66,O*f67_69,O*f70_74,O*f75_79,O*f80_84,O*f85+,O*homevalue,tu5,t5_9,t10_14,t15_17,t18_19,t20,t21,t22_24,t25_29,t30_34,t35_39,t40_44,t45_49,t50_54,t55_59,t60_61,t62_64,t65_66,t67_69,t70_74,t75_79,t80_84,t85+,O*tu5,O*t5_9,O*t10_14,O*t15_17,O*t18_19,O*t20,O*t21,O*t22_24,O*t25_29,O*t30_34,O*t35_39,O*t40_44,O*t45_49,O*t50_54,O*t55_59,O*t60_61,O*t62_64,O*t65_66,O*t67_69,O*t70_74,O*t75_79,O*t80_84,O*t85+,child,O*child,taxbase,O*taxbase,O65,O*O65,Pchild,Ptaxbase,PO65,O*Pchild,O*Ptaxbase,O*PO65
0,"Big Stone Gap, VA Micro Area",310M500US13720,42456,22025,1160,1149,1253,776,583,356,238,1035,1791,1490,1432,1480,1430,1388,1465,768,853,425,861,858,559,340,335,20431,1054,1297,1012,702,447,345,172,606,1360,1143,1272,1123,1317,1248,1611,596,1047,740,646,947,638,573,535,86200,60499,30657,1614,1942,1599,1070,826,443,333,1527,2149,2257,1918,2250,2176,2222,1710,959,1513,668,951,1034,755,430,311,29842,1605,1674,1795,1076,801,521,484,1008,1650,1667,1911,1888,2065,2158,1961,834,1569,605,860,1347,1013,646,704,82800,2214,2446,2265,1478,1030,701,410,1641,3151,2633,2704,2603,2747,2636,3076,1364,1900,1165,1507,1805,1197,913,870,3219,3616,3394,2146,1627,964,817,2535,3799,3924,3829,4138,4241,4380,3671,1793,3082,1273,1811,2381,1768,1076,1015,8403,12375,26596,38800,7457,9324,19.8,62.6,17.6,20.5,64.1,15.4
1,"Billings, MT Metro Area",310M500US13740,179071,88433,5510,6170,5862,3575,1936,774,904,3687,5856,5880,5701,5403,5355,5446,6014,2780,3459,2176,3299,3299,2395,1420,1532,90638,5301,5588,6219,3210,1857,743,944,3251,5905,6083,5596,5425,5204,5681,6679,2569,3502,2014,3357,4053,2710,2050,2697,227900,162892,79941,5415,5422,5184,3152,2033,993,1071,2735,5529,5384,4883,4960,5268,5976,5848,2202,2950,1611,2004,2704,1942,1586,1089,82951,5205,5494,5052,3050,1722,1018,922,3161,5581,5401,4971,4769,5403,6224,5813,2420,3039,1952,2190,2776,2276,1989,2523,187100,10811,11758,12081,6785,3793,1517,1848,6938,11761,11963,11297,10828,10559,11127,12693,5349,6961,4190,6656,7352,5105,3470,4229,10620,10916,10236,6202,3755,2011,1993,5896,11110,10785,9854,9729,10671,12200,11661,4622,5989,3563,4194,5480,4218,3575,3612,41435,37974,106634,100276,31002,24642,23.1,59.5,17.3,23.3,61.6,15.1


Make a separate dataframe of only these created groups and their shares

In [243]:
data = data[['CBSA', 'GEOID', 'total', 'child', 'taxbase', 'O65', 
           'Pchild', 'Ptaxbase', 'PO65', 'homevalue', 'O*total','O*child', 'O*taxbase', 'O*O65', 
           'O*Pchild', 'O*Ptaxbase', 'O*PO65', 'O*homevalue']]

In [244]:
data.head()

,CBSA,GEOID,total,child,taxbase,O65,Pchild,Ptaxbase,PO65,homevalue,O*total,O*child,O*taxbase,O*O65,O*Pchild,O*Ptaxbase,O*PO65,O*homevalue
0,"Big Stone Gap, VA Micro Area",310M500US13720,42456,8403,26596,7457,19.8,62.6,17.6,86200,60499,12375,38800,9324,20.5,64.1,15.4,82800
1,"Billings, MT Metro Area",310M500US13740,179071,41435,106634,31002,23.1,59.5,17.3,227900,162892,37974,100276,24642,23.3,61.6,15.1,187100
2,"Binghamton, NY Metro Area",310M500US13780,241874,47899,148264,45711,19.8,61.3,18.9,117600,249261,50798,156323,42140,20.4,62.7,16.9,108800
3,"Birmingham-Hoover, AL Metro Area",310M500US13820,1085330,250498,666052,168780,23.1,61.4,15.6,165200,1135534,267614,712019,155901,23.6,62.7,13.7,144000
4,"Bismarck, ND Metro Area",310M500US13900,127503,29171,78158,20174,22.9,61.3,15.8,249000,120688,27966,75829,16893,23.2,62.8,14.0,171000


Let's split the full GEOID so that we have the CBSA FIPS, let's also split up the CBSA names so that we have the name, state, and Metro or Micro Area designation. Start with the geography name.

In [245]:
# Split the CBSA column at the comma then name the new column as the first column you index into
name = data['CBSA'].str.split(pat = ',', expand = True)
name

,0,1
0,Big Stone Gap,VA Micro Area
1,Billings,MT Metro Area
2,Binghamton,NY Metro Area
3,Birmingham-Hoover,AL Metro Area
4,Bismarck,ND Metro Area
...,...,...
854,Opelousas,LA Micro Area
855,Orangeburg,SC Micro Area
856,Orlando-Kissimmee-Sanford,FL Metro Area
857,Fort Dodge,IA Micro Area


In [246]:
data['Name'] = name[0]
state_area = name[1]
state_area

0       VA Micro Area
1       MT Metro Area
2       NY Metro Area
3       AL Metro Area
4       ND Metro Area
            ...      
854     LA Micro Area
855     SC Micro Area
856     FL Metro Area
857     IA Micro Area
858     CO Metro Area
Name: 1, Length: 859, dtype: object

In [247]:
# Next we can split this again after the first two figures which indicate the State
state = state_area.str.slice(stop = 3)
state

0       VA
1       MT
2       NY
3       AL
4       ND
      ... 
854     LA
855     SC
856     FL
857     IA
858     CO
Name: 1, Length: 859, dtype: object

In [248]:
data['State'] = state

In [249]:
area = state_area.str.slice(start = 3, stop = 9)
area

0       Micro
1       Metro
2       Metro
3       Metro
4       Metro
        ...  
854     Micro
855     Micro
856     Metro
857     Micro
858     Metro
Name: 1, Length: 859, dtype: object

In [250]:
data['Area'] = area

Now we're going to split the GEOIDs.

In [251]:
data['CBSAFIPS'] = data['GEOID'].str.slice(start = 9)

In [252]:
data.head()

,CBSA,GEOID,total,child,taxbase,O65,Pchild,Ptaxbase,PO65,homevalue,O*total,O*child,O*taxbase,O*O65,O*Pchild,O*Ptaxbase,O*PO65,O*homevalue,Name,State,Area,CBSAFIPS
0,"Big Stone Gap, VA Micro Area",310M500US13720,42456,8403,26596,7457,19.8,62.6,17.6,86200,60499,12375,38800,9324,20.5,64.1,15.4,82800,Big Stone Gap,VA,Micro,13720
1,"Billings, MT Metro Area",310M500US13740,179071,41435,106634,31002,23.1,59.5,17.3,227900,162892,37974,100276,24642,23.3,61.6,15.1,187100,Billings,MT,Metro,13740
2,"Binghamton, NY Metro Area",310M500US13780,241874,47899,148264,45711,19.8,61.3,18.9,117600,249261,50798,156323,42140,20.4,62.7,16.9,108800,Binghamton,NY,Metro,13780
3,"Birmingham-Hoover, AL Metro Area",310M500US13820,1085330,250498,666052,168780,23.1,61.4,15.6,165200,1135534,267614,712019,155901,23.6,62.7,13.7,144000,Birmingham-Hoover,AL,Metro,13820
4,"Bismarck, ND Metro Area",310M500US13900,127503,29171,78158,20174,22.9,61.3,15.8,249000,120688,27966,75829,16893,23.2,62.8,14.0,171000,Bismarck,ND,Metro,13900


# Now we start thinking about the kinds of change we want to look at.

With the measures I think the two most sensical ways to look at things are: percent change in the raw numbers, and change in population share.

First let's create the percent change:

In [253]:
data['O65percchange'] = round((data['O65'] - data['O*O65'])*100/data['O*O65'], 2)
data['taxbasepercchange'] = round((data['taxbase'] - data['O*taxbase'])*100/data['O*taxbase'], 2)
data['childpercchange'] = round((data['child'] - data['O*child'])*100/data['O*child'], 2)
data['totalpercchange'] = round((data['total']-data['O*total'])*100/data['O*total'], 2)
data['homepercchange'] = round((data['homevalue']-data['O*homevalue'])*100/data['O*homevalue'], 2)

Now change in share and real change:

In [254]:
data['O65sharechange'] = data['PO65'] - data['O*PO65']
data['taxbasesharechange'] = data['Ptaxbase'] - data['O*Ptaxbase']
data['childsharechange'] = data['Pchild'] - data['O*Pchild']
data['O65realchange'] = data['O65'] - data['O*O65']
data['taxbaserealchange'] = data['taxbase'] - data['O*taxbase']
data['childrealchange'] = data['child'] - data['O*child']
data['totalrealchange']= data['total'] - data['O*total']
data['homerealchange'] = data['homevalue'] - data['O*homevalue']

In [255]:
data.head()

,CBSA,GEOID,total,child,taxbase,O65,Pchild,Ptaxbase,PO65,homevalue,O*total,O*child,O*taxbase,O*O65,O*Pchild,O*Ptaxbase,O*PO65,O*homevalue,Name,State,Area,CBSAFIPS,O65percchange,taxbasepercchange,childpercchange,totalpercchange,homepercchange,O65sharechange,taxbasesharechange,childsharechange,O65realchange,taxbaserealchange,childrealchange,totalrealchange,homerealchange
0,"Big Stone Gap, VA Micro Area",310M500US13720,42456,8403,26596,7457,19.8,62.6,17.6,86200,60499,12375,38800,9324,20.5,64.1,15.4,82800,Big Stone Gap,VA,Micro,13720,-20.02,-31.45,-32.10,-29.82,4.11,2.2,-1.5,-0.7,-1867,-12204,-3972,-18043,3400
1,"Billings, MT Metro Area",310M500US13740,179071,41435,106634,31002,23.1,59.5,17.3,227900,162892,37974,100276,24642,23.3,61.6,15.1,187100,Billings,MT,Metro,13740,25.81,6.34,9.11,9.93,21.81,2.2,-2.1,-0.2,6360,6358,3461,16179,40800
2,"Binghamton, NY Metro Area",310M500US13780,241874,47899,148264,45711,19.8,61.3,18.9,117600,249261,50798,156323,42140,20.4,62.7,16.9,108800,Binghamton,NY,Metro,13780,8.47,-5.16,-5.71,-2.96,8.09,2.0,-1.4,-0.6,3571,-8059,-2899,-7387,8800
3,"Birmingham-Hoover, AL Metro Area",310M500US13820,1085330,250498,666052,168780,23.1,61.4,15.6,165200,1135534,267614,712019,155901,23.6,62.7,13.7,144000,Birmingham-Hoover,AL,Metro,13820,8.26,-6.46,-6.40,-4.42,14.72,1.9,-1.3,-0.5,12879,-45967,-17116,-50204,21200
4,"Bismarck, ND Metro Area",310M500US13900,127503,29171,78158,20174,22.9,61.3,15.8,249000,120688,27966,75829,16893,23.2,62.8,14.0,171000,Bismarck,ND,Metro,13900,19.42,3.07,4.31,5.65,45.61,1.8,-1.5,-0.3,3281,2329,1205,6815,78000


In [256]:
#index into where the Nashville MSA is to check the rankings out
nash = data.loc[data['CBSA'] == 'Nashville-Davidson--Murfreesboro--Franklin, TN Metro Area'].reset_index(drop = True)

In [257]:
print("The following numbers are Nashville's percent growth in a 5 year over 5 year period for total elderly population, then the share change between the 2014 and 2019 periods:")
print('______________________________________________________________________________')
print(nash['O65percchange'])
print(nash['O65sharechange'])
print("The following number are Nashville's percent growth in a 5 year over 5 year period for total tax base population, then the share change between the 2014 and 2019 periods:")
print('______________________________________________________________________________')
print(nash['taxbasepercchange'])
print(nash['taxbasesharechange'])
print("The following number are Nashville's percent growth in a 5 year over 5 year period for total child population, then the share change between the 2014 and 2019 periods:")
print('______________________________________________________________________________')
print(nash['childpercchange'])
print(nash['childsharechange'])

The following numbers are Nashville's percent growth in a 5 year over 5 year period for total elderly population, then the share change between the 2014 and 2019 periods:
______________________________________________________________________________
0    21.72
Name: O65percchange, dtype: float64
0    1.4
Name: O65sharechange, dtype: float64
The following number are Nashville's percent growth in a 5 year over 5 year period for total tax base population, then the share change between the 2014 and 2019 periods:
______________________________________________________________________________
0    6.97
Name: taxbasepercchange, dtype: float64
0   -0.7
Name: taxbasesharechange, dtype: float64
The following number are Nashville's percent growth in a 5 year over 5 year period for total child population, then the share change between the 2014 and 2019 periods:
______________________________________________________________________________
0    4.9
Name: childpercchange, dtype: float64
0   -0.7
Name

In [258]:
data.corr()

,total,child,taxbase,O65,Pchild,Ptaxbase,PO65,homevalue,O*total,O*child,O*taxbase,O*O65,O*Pchild,O*Ptaxbase,O*PO65,O*homevalue,O65percchange,taxbasepercchange,childpercchange,totalpercchange,homepercchange,O65sharechange,taxbasesharechange,childsharechange,O65realchange,taxbaserealchange,childrealchange,totalrealchange,homerealchange
total,1.000000,0.996776,0.999834,0.991324,0.022526,0.139218,-0.135573,0.296189,0.998979,0.998252,0.998779,0.986379,0.042132,0.112103,-0.139169,0.293595,0.052483,0.066643,0.024503,0.051167,0.116818,-0.043974,0.119169,-0.082522,0.968627,-0.007367,-0.513270,0.263047,0.231464
child,0.996776,1.000000,0.996252,0.979604,0.048907,0.137094,-0.154529,0.289851,0.993515,0.998685,0.992831,0.972490,0.068566,0.107278,-0.157473,0.284507,0.055544,0.072750,0.030620,0.055734,0.126077,-0.054973,0.130312,-0.081918,0.971172,0.042826,-0.466698,0.310093,0.233671
taxbase,0.999834,0.996252,1.000000,0.990267,0.021029,0.144786,-0.139099,0.298241,0.998800,0.997814,0.998884,0.985251,0.040363,0.117735,-0.142825,0.295766,0.052573,0.066070,0.024230,0.050462,0.115428,-0.044967,0.119159,-0.081420,0.968073,-0.006094,-0.514494,0.263273,0.232720
O65,0.991324,0.979604,0.990267,1.000000,-0.012608,0.116152,-0.088448,0.292854,0.993884,0.985059,0.993200,0.998838,0.007815,0.093388,-0.092449,0.294092,0.046532,0.058610,0.015753,0.046355,0.106695,-0.021740,0.100014,-0.087102,0.953056,-0.091539,-0.573517,0.184483,0.219154
Pchild,0.022526,0.048907,0.021029,-0.012608,1.000000,-0.247675,-0.577160,-0.167953,0.019360,0.043370,0.017047,-0.013836,0.972490,-0.348479,-0.516919,-0.212468,-0.028241,0.136117,0.168543,0.078346,0.135155,-0.504359,0.408130,0.148330,-0.004212,0.083694,0.071602,0.073040,-0.013905
Ptaxbase,0.139218,0.137094,0.144786,0.116152,-0.247675,1.000000,-0.648146,0.194061,0.135791,0.133677,0.140940,0.111705,-0.248103,0.971491,-0.679837,0.205510,0.164646,0.097582,0.064448,0.059199,-0.001729,-0.149379,0.164225,-0.005996,0.137799,0.077135,-0.008126,0.106766,0.118100
PO65,-0.135573,-0.154529,-0.139099,-0.088448,-0.577160,-0.648146,1.000000,-0.031886,-0.130172,-0.147281,-0.132698,-0.083713,-0.555172,-0.544866,0.979174,-0.006506,-0.116364,-0.189389,-0.187018,-0.111581,-0.105074,0.523143,-0.459284,-0.111553,-0.113401,-0.131409,-0.049434,-0.148038,-0.088970
homevalue,0.296189,0.289851,0.298241,0.292854,-0.167953,0.194061,-0.031886,1.000000,0.291279,0.287403,0.293168,0.284577,-0.138348,0.235053,-0.096440,0.974923,0.346336,0.220914,0.176982,0.243834,0.368569,0.257087,-0.161708,-0.130979,0.328978,0.098560,-0.099603,0.176307,0.823115
O*total,0.998979,0.993515,0.998800,0.993884,0.019360,0.135791,-0.130172,0.291279,1.000000,0.997177,0.999862,0.990962,0.038730,0.109283,-0.133647,0.290857,0.043099,0.055482,0.013879,0.040037,0.106675,-0.042131,0.116487,-0.081621,0.958339,-0.052166,-0.549391,0.219190,0.222193
O*child,0.998252,0.998685,0.997814,0.985059,0.043370,0.133677,-0.147281,0.287403,0.997177,1.000000,0.996588,0.979977,0.063552,0.104308,-0.150422,0.284042,0.046059,0.060872,0.018018,0.043803,0.116073,-0.050987,0.128294,-0.084309,0.963563,-0.003702,-0.511418,0.263769,0.226751


### Export

I'm going to export here and we can explore the data further in a new notebook - I want to separate prep for simplicity's sake.

In [259]:
data.to_csv('../output/csv/agegroupshomevalue_2014_2019_cbsa.csv', index = False)